<a href="https://colab.research.google.com/github/dangdang2222/frankmocap_project/blob/main/3D_keypoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 고쳐야할거
1. csv 읽어오는 경로
2. 카메라 개수(for문으로 싹 수정해야함)
3. csv 저장하는 경로


In [2]:
import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

column_idx = ['Nose_x','Nose_y','Nose_z','LEye_x','LEye_y','LEye_z','REye_x','REye_y','REye_z','LEar_x','LEar_y','LEar_z',
           'REar_x','REar_y','REar_z','LShoulder_x','LShoulder_y','LShoulder_z','RShoulder_x','RShoulder_y','RShoulder_z',
           'LElbow_x','LElbow_y','LElbow_z','RElbow_x','RElbow_y','RElbow_z','LWrist_x','LWrist_y','LWrist_z',
           'RWrist_x','RWrist_y','RWrist_z','LHip_x','LHip_y','LHip_z','RHip_x','RHip_y','RHip_z','LKnee_x','LKnee_y','LKnee_z',
           'RKnee_x','RKnee_y','RKnee_z','LAnkle_x','LAnkle_y','LAnkle_z','RAnkle_x','RAnkle_y','RAnkle_z','Head_x','Head_y','Head_z',
           'Neck_x','Neck_y','Neck_z','Hip_x','Hip_y','Hip_z','LBigToe_x','LBigToe_y','LBigToe_z','RBigToe_x','RBigToe_y','RBigToe_z',
           'LSmallToe_x','LSmallToe_y','LSmallToe_z','RSmallToe_x','RSmallToe_y','RSmallToe_z','LHeel_x','LHeel_y','LHeel_z','RHeel_x','RHeel_y','RHeel_z']

K = np.array([[1050.74627,0,959.5],[0,1052.54163,539.5],[0,0,1]])

Rt1 = np.array([[-0.060679, 0.998130, 0.007343,-605.413346],[0.418569, 0.032123, -0.907617, 548.137858],[-0.906156,-0.052000,-0.419735,4336.592638]])
P1 = np.dot(K,Rt1)
Rt2 = np.array([[ -0.066097,-0.997812,0.001260,653.352810],[-0.432199,0.027492,-0.901359,1175.918886],[0.899353 ,-0.060121,-0.433071,3114.943323]])
P2 = np.dot(K,Rt2)
Rt3 = np.array([[ 0.784980,-0.619352,-0.014461,-420.172604],[-0.276741,-0.329672,-0.902624,295.490400],[0.554275,0.712544,-0.430186 ,4881.382220]])
P3 = np.dot(K,Rt3)
Rt4 = np.array([[-0.999920,-0.012570,-0.001528,728.642753],[-0.003610,0.398618,-0.917110,7.551169],[0.012137,-0.917031,-0.398631,5699.942529]])
P4 = np.dot(K,Rt4)
Rt5 = np.array([[-0.733829,-0.677994,-0.042651,1966.137295],[-0.217095,0.293539,-0.930970,76.021013],[0.643711,-0.673913,-0.362597,5872.706559]])
P5 = np.dot(K,Rt5)
Rt6 = np.array([[0.799458,0.600171,-0.025722,-704.819635],[0.247259,-0.367782,-0.896437,-127.891999],[-0.547476,0.710304,-0.442424,5625.110129]])
P6 = np.dot(K,Rt6)
Rt7 = np.array([[-0.755252,0.655282,0.014137,-737.902102],[0.279717,0.341747,-0.897200,-790.877554],[-0.592750,-0.673657,-0.441399,6746.539080]])
P7 = np.dot(K,Rt7)
Rt8 = np.array([[0.999871,0.015922,0.002158,-621.529759],[0.008548,-0.413380,-0.910518,660.889110],[-0.013605,0.910419,-0.413463,4254.063740]])
P8 = np.dot(K,Rt8)

P = [P1, P2, P3, P4, P5, P6, P7, P8]

def triangulate_nviews(P, ip):
    if not len(ip) == len(P):
        raise ValueError('Number of points and number of cameras not equal.')
    n = len(P)
    M = np.zeros([3*n, 4+n])
    for i, (x, p) in enumerate(zip(ip, P)):
        M[3*i:3*i+3, :4] = p
        M[3*i:3*i+3, 4+i] = -x
    V = np.linalg.svd(M)[-1]
    X = V[-1, :4]
    X /= X[3]
    return X[:3]

df_image = []
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera0/local_keypoints/Motion2-1 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera1/local_keypoints/Motion2-2 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera2/local_keypoints/Motion2-3 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera3/local_keypoints/Motion2-4 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera4/local_keypoints/Motion2-5 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera5/local_keypoints/Motion2-6 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera6/local_keypoints/Motion2-7 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image.append(pd.read_csv('/content/drive/MyDrive/2D Skeleton(CSV), JPG/camera7/local_keypoints/Motion2-8 - 1of4.csv').drop(['image_filename'],axis=1).to_numpy(dtype='float32'))
df_image = np.array(df_image) #(8,301,52)

In [3]:
temp = np.zeros((301,8,26,2))

for frame_idx in range(301):
  for camera_idx in range(8):
    for point_idx in range(26):
      temp[frame_idx][camera_idx][point_idx][0] = df_image[camera_idx][frame_idx][2*point_idx]
      temp[frame_idx][camera_idx][point_idx][1] = df_image[camera_idx][frame_idx][2*point_idx+1]
arr = np.array(temp)

In [4]:
D = []
P = [P1, P2, P3, P4, P5, P6, P7, P8]
for frame_idx in range(301):
  d = []
  for point in range(0,26):
    original = []
    for i in range(8):
      original.append(np.concatenate((arr[frame_idx][i][point],[1.0])))
    X = triangulate_nviews(P, original)
    d.append(X)

  D.append(d)
D = np.array(D)

df = pd.DataFrame(D.reshape((301,26*3)),columns = column_idx)
df.to_csv('3D_points.csv')